# Tensorflow

In [1]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist

# 载入 MNIST 手写阿拉伯数字资料
(x_train, y_train),(x_test, y_test) = mnist.load_data()

# 特征缩放，使用常态化(Normalization)，公式 = (x - min) / (max - min)
x_train_norm, x_test_norm = x_train / 255.0, x_test / 255.0

# 转为 Dataset，含 X/Y 资料
train_ds = tf.data.Dataset.from_tensor_slices((x_train_norm, y_train))

In [2]:
# 建立模型
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

In [3]:
# 设定优化器(optimizer)、损失函数(loss)、效能衡量指标(metrics)的类别
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## 或是 one-hot encoding

In [ ]:
# 将 training 的 label 进行 one-hot encoding，例如数字 7 经过 One-hot encoding 转换后是 0000000100，即第8个值为 1
y_train = tf.keras.utils.to_categorical(y_train) 
y_test = tf.keras.utils.to_categorical(y_test) 
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## 模型训练

In [4]:
model.fit(x_train_norm, y_train, epochs=5, validation_split=0.2)

Epoch 1/5
1500/1500 [==============================] - 6s 3ms/step - loss: 0.3214 - accuracy: 0.9073 - val_loss: 0.1567 - val_accuracy: 0.9563
Epoch 2/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.1565 - accuracy: 0.9538 - val_loss: 0.1150 - val_accuracy: 0.9671
Epoch 3/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.1170 - accuracy: 0.9647 - val_loss: 0.1005 - val_accuracy: 0.9718
Epoch 4/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0946 - accuracy: 0.9717 - val_loss: 0.0934 - val_accuracy: 0.9734
Epoch 5/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0804 - accuracy: 0.9747 - val_loss: 0.0839 - val_accuracy: 0.9753


In [5]:
# 评分(Score Model)
score=model.evaluate(x_test_norm, y_test, verbose=0)

for i, x in enumerate(score):
    print(f'{model.metrics_names[i]}: {score[i]:.4f}')

loss: 0.0801
accuracy: 0.9750


# PyTorch

In [6]:
import os
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchmetrics import Accuracy
from torchvision import transforms
from torchvision.datasets import MNIST

In [8]:
PATH_DATASETS = "" # 预设路径
# 下载 MNIST 手写阿拉伯数字 训练资料
train_ds = MNIST(PATH_DATASETS, train=True, download=True, 
                 transform=transforms.ToTensor())

# 下载测试资料
test_ds = MNIST(PATH_DATASETS, train=False, download=True, 
                 transform=transforms.ToTensor())

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
"cuda" if torch.cuda.is_available() else "cpu"

'cuda'

In [9]:
# 建立模型
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(28 * 28, 256), 
    torch.nn.Dropout(0.2),
    torch.nn.Linear(256, 10), 
    # 使用nn.CrossEntropyLoss()时，不需要将输出经过softmax层，否则计算的损失会有误
    # torch.nn.Softmax(dim=1)
).to(device)

## 模型训练

In [ ]:
epochs = 5
lr=0.1
BATCH_SIZE = 1024  # 批量

# 建立 DataLoader
train_loader = DataLoader(train_ds, batch_size=600)

# 设定优化器(optimizer)
# optimizer = torch.optim.Adam(model.parameters(), lr=lr)
optimizer = torch.optim.Adadelta(model.parameters(), lr=lr)

criterion = nn.CrossEntropyLoss()

In [10]:
model.train()
loss_list = []    
for epoch in range(1, epochs + 1):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
#         if batch_idx == 0 and epoch == 1: print(data[0])

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 10 == 0:
            loss_list.append(loss.item())
            batch = batch_idx * len(data)
            data_count = len(train_loader.dataset)
            percentage = (100. * batch_idx / len(train_loader))
            print(f'Epoch {epoch}: [{batch:5d} / {data_count}] ({percentage:.0f} %)' +
                  f'  Loss: {loss.item():.6f}')

Epoch 1: [    0 / 60000] (0 %)  Loss: 2.321503
Epoch 1: [ 6000 / 60000] (10 %)  Loss: 2.024913
Epoch 1: [12000 / 60000] (20 %)  Loss: 1.808975
Epoch 1: [18000 / 60000] (30 %)  Loss: 1.580163
Epoch 1: [24000 / 60000] (40 %)  Loss: 1.356292
Epoch 1: [30000 / 60000] (50 %)  Loss: 1.239593
Epoch 1: [36000 / 60000] (60 %)  Loss: 1.009667
Epoch 1: [42000 / 60000] (70 %)  Loss: 0.977577
Epoch 1: [48000 / 60000] (80 %)  Loss: 0.717749
Epoch 1: [54000 / 60000] (90 %)  Loss: 0.738873
Epoch 2: [    0 / 60000] (0 %)  Loss: 0.692975
Epoch 2: [ 6000 / 60000] (10 %)  Loss: 0.545819
Epoch 2: [12000 / 60000] (20 %)  Loss: 0.643318
Epoch 2: [18000 / 60000] (30 %)  Loss: 0.552558
Epoch 2: [24000 / 60000] (40 %)  Loss: 0.576016
Epoch 2: [30000 / 60000] (50 %)  Loss: 0.625852
Epoch 2: [36000 / 60000] (60 %)  Loss: 0.511515
Epoch 2: [42000 / 60000] (70 %)  Loss: 0.613690
Epoch 2: [48000 / 60000] (80 %)  Loss: 0.421063
Epoch 2: [54000 / 60000] (90 %)  Loss: 0.463095
Epoch 3: [    0 / 60000] (0 %)  Loss: 0.46

In [11]:
# 建立 DataLoader
test_loader = DataLoader(test_ds, shuffle=False, batch_size=BATCH_SIZE)

model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        
        # sum up batch loss
        test_loss += criterion(output, target).item()
        
        # 预测
        pred = output.argmax(dim=1, keepdim=True)  
        
        # 正确笔数
        correct += pred.eq(target.view_as(pred)).sum().item()

# 平均损失
test_loss /= len(test_loader.dataset)
# 显示测试结果
batch = batch_idx * len(data)
data_count = len(test_loader.dataset)
percentage = 100. * correct / data_count
print(f'平均损失: {test_loss:.4f}, 准确率: {correct}/{data_count}' + 
      f' ({percentage:.0f}%)\n')

平均损失: 0.0003, 准确率: 9059/10000 (91%)

